In [1]:
import jieba
import pickle
import math
import csv
from collections import defaultdict

# 增加字段大小限制，设置为更大的值（比如 10MB）
csv.field_size_limit(10000000)

# 读取倒排索引
def load_index(filename="index.pkl"):
    print("加载倒排索引...")
    with open(filename, 'rb') as f:
        inverted_index, doc_tfs, idf = pickle.load(f)
    print("倒排索引加载完成。")
    return inverted_index, doc_tfs, idf

# 中文分词
def tokenize(text):
    return list(jieba.cut(text))

# 计算查询的TF-IDF向量
def compute_query_tfidf(query, idf):
    query_tokens = tokenize(query)
    query_tf = defaultdict(int)
    
    # 计算TF
    for word in query_tokens:
        query_tf[word] += 1
    total_words = len(query_tokens)
    for word in query_tf:
        query_tf[word] /= total_words
    
    # 计算TF-IDF
    query_tfidf = {}
    for word, tf in query_tf.items():
        query_tfidf[word] = tf * idf.get(word, 0)  # 默认IDF为0
    
    return query_tfidf

# 计算文档与查询的相关度（余弦相似度）
def compute_cosine_similarity(query_tfidf, doc_tfidf):
    dot_product = sum(query_tfidf.get(word, 0) * doc_tfidf.get(word, 0) for word in query_tfidf)
    query_norm = math.sqrt(sum(val**2 for val in query_tfidf.values()))
    doc_norm = math.sqrt(sum(val**2 for val in doc_tfidf.values()))
    if query_norm * doc_norm == 0:
        return 0
    return dot_product / (query_norm * doc_norm)

# 执行搜索
def search(query, inverted_index, doc_tfs, idf, top_n=10):
    print(f"正在搜索：{query}")
    # 计算查询的TF-IDF向量
    query_tfidf = compute_query_tfidf(query, idf)
    
    # 计算每个文档与查询的相关度
    scores = []
    for doc_id, doc_tfidf in enumerate(doc_tfs):
        score = compute_cosine_similarity(query_tfidf, doc_tfidf)
        scores.append((doc_id, score))
    
    # 按相关度排序并返回前top_n个文档
    scores.sort(key=lambda x: x[1], reverse=True)
    top_docs = scores[:top_n]
    
    return top_docs

In [2]:
# 打开并读取 HtmlFile.csv 文件
csv_filename = "HtmlContent.csv"

# 读取文件中的内容
with open(csv_filename, mode='r', encoding='utf-8') as file:
    csv_reader = csv.reader(file)
    # 跳过表头（如果有的话）
    next(csv_reader, None)
    # 将所有内容读入列表
    html_data = list(csv_reader)

# 加载倒排索引
inverted_index, doc_tfs, idf = load_index()

加载倒排索引...
倒排索引加载完成。


In [4]:
while True:
    # 用户输入查询
    query = input("请输入查询词（输入 'exit' 退出）：")
    
    # 如果用户输入 "exit"，退出循环
    if query.lower() == 'exit':
        print("退出程序。")
        break
    
    # 执行搜索
    top_docs = search(query, inverted_index, doc_tfs, idf)
    
    # 输出搜索结果
    print("搜索结果：")
    for doc_id, score in top_docs:
        # 获取网页的标题和 URL
        webpage_title = html_data[doc_id][0]  # 假设第一列是标题
        webpage_url = html_data[doc_id][1]    # 假设第二列是 URL
        # 输出结果
        print(f"网页序号: {doc_id + 1}, 网页标题: {webpage_title}, 网页链接: {webpage_url}, 网页相关度: {score:.4f}")

正在搜索：doc
搜索结果：
网页序号: 78167, 网页标题: 河北省财政厅关于印发《河北省高层次财会人才选拔培养方案》的通知, 网页链接: https://czt.hebei.gov.cn/root17/zfxx/202411/t20241112_2103961.html, 网页相关度: 0.0768
网页序号: 75925, 网页标题: 杭州市人民政府门户-网上听证, 网页链接: https://minyi.zjzwfw.gov.cn/dczjnewls/dczj/idea/topic_19291.html, 网页相关度: 0.0547
网页序号: 80066, 网页标题: 杭州市人民政府门户-网上听证, 网页链接: https://minyi.zjzwfw.gov.cn/dczjnewls/dczj/idea/topic_18814.html, 网页相关度: 0.0547
网页序号: 88046, 网页标题: 交通运输部关于公布第70批道路运输车辆达标车型的公告-政府信息公开-交通运输部, 网页链接: https://xxgk.mot.gov.cn/2020/jigou/ysfws/202412/t20241217_4161190.html, 网页相关度: 0.0508
网页序号: 84810, 网页标题: 行政许可_        表格下载_        福建省统计局, 网页链接: https://tjj.fujian.gov.cn/bsdt/bgxz/xzxk/201905/t20190515_4875880.htm, 网页相关度: 0.0472
网页序号: 20568, 网页标题: 保卫处举办消防安全知识宣传教育活动-综合新闻-南开大学, 网页链接: http://news.nankai.edu.cn/zhxw/system/2010/07/13/000032521.shtml, 网页相关度: 0.0426
网页序号: 84286, 网页标题: 交通运输部关于印发《道路运输企业主要负责人和安全生产管理人员安全考核管理办法》《道路运输企业主要负责人和安全生产管理人员安全考核大纲》的通知_政府信息公开_安徽省交通运输厅, 网页链接: https://jtt.ah.gov.cn/public/21701/122527401.html, 网页相关度: 0.